In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(temperature=0.9)

In [7]:
from langchain import PromptTemplate


template = """
I want you to act as a naming consultant for new companies.

Here are some examples of good company names:

- search engine, Google
- social media, Facebook
- video sharing, YouTube

The name should be short, catchy and easy to remember.

What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("personality tests")

AuthenticationError: Incorrect API key provided: sk-02IFn***************************************oLy8. You can find your API key at https://platform.openai.com/account/api-keys.